In [35]:
import os
import sqlite3
import pandas as pd
import numpy as np

**Nettoyage des données**

In [36]:
notes = ['C','D','E','F','G','A','B']
demi_ton = ['C','C#','D','D#','E','F','F#','G','G#','A','A#','B']
demi_ton_b = ['C','Db','D','Eb','E','F','Gb','G','Ab','A','Bb','B']

In [37]:
def get_fund(liste):
    chords_cut = []
    for i in range(len(liste)):
        if ":" in liste[i]:
           chords_cut.append(liste[i].split(':')[0])
        else: 
            chords_cut.append(liste[i].split('/')[0])

    return chords_cut

In [38]:
def get_type(liste):
    chords_cut = [] 
    for i in range(len(liste)):
        val = liste[i]
        temp = 'M'
        if ":" in liste[i]:
           temp = liste[i].split(':')[1]
           if "/" in temp:
               chords_cut.append(temp.split('/')[0])
           else:
               chords_cut.append(temp)
        else: 
            chords_cut.append(temp)

    return chords_cut

In [39]:
def get_basse_mod(liste):
    chords_cut = []
    for i in range(len(liste)):
        if ":" in liste[i]:
            temp = liste[i].split(':')[1]
            if "/" in temp:
                chords_cut.append(temp.split('/')[1])
            else:
                chords_cut.append(' ')
        else: 
            if "/" in liste[i]:
                chords_cut.append(liste[i].split('/')[1])
            else: 
                chords_cut.append(' ')

    return chords_cut

In [40]:
dict_intervalle = {'b2': 1, '2': 1, 'b3': 3, '3': 3, 'b4': 4, '4': 4, 'b5': 5, '5': 5, 'b6': 6, '6': 6, 'b7': 7, '7': 7}
dict_ton = {'b2': 0.5, '2': 1, 'b3': 1.5, '3': 2, 'b4': 2.5, '4': 3, 'b5': 3, '5': 3.5, 'b6': 4, '6': 4.5, 'b7': 5, '7': 5}

In [41]:
def get_basse_inter(liste):
    key_to_find = liste[2]
    if key_to_find in dict_intervalle:
        intervalle = dict_intervalle[key_to_find]

        fund = liste[0]
        if len(fund) >=2:
            fund = fund[0]

        idx = notes.index(fund)
        idx_basse = idx + intervalle
        if idx_basse >= 7:
            idx_basse -= 7
        inter_basse = notes[idx_basse-1]
    else:
        return 'No basse modification'
    return inter_basse

In [42]:
def get_basse_ton(liste):
    key_to_find = liste[2]
    fund_basse = ''
    
    if key_to_find in dict_ton:
        val_ton = dict_ton[key_to_find]*2
        idx = demi_ton.index(liste[0])
        idx_basse = idx + val_ton
        if idx_basse >= 12:
            idx_basse -= 12
        inter_basse_ton = demi_ton[int(idx_basse)]

        note_inter = get_basse_inter(liste)

        if len(inter_basse_ton) >=2:
            fund_basse = inter_basse_ton[0]
        
        if note_inter != fund_basse:
            inter_basse_ton = demi_ton_b[int(idx_basse)]

    else:
        inter_basse_ton = ''

   
    return inter_basse_ton

In [43]:
def concat(lis1,lis2,lis3):
    total = []
    for i in range(len(lis1)):
        total.append([lis1[i],lis2[i],lis3[i]])
    return total

In [44]:
def pretitify(list):
    res = []
    for value in list:
        if value[2] != ' ':
            res.append(value[0]+value[1]+'/'+ get_basse_ton(value))
        else:
            res.append(value[0]+value[1])
    return res

**Creéation d'un df avec les données**

In [45]:
# Spécifiez le chemin vers le dossier contenant le fichier LAB
dossier = r"C:\Users\lucie\Documents\Beatles\Please Please Me"

# Obtenez la liste des fichiers dans le dossier
fichiers = [f for f in os.listdir(dossier) if os.path.isfile(os.path.join(dossier, f))]

# Initialisez un DataFrame vide
df_final = pd.DataFrame()

# Parcourez chaque fichier dans le dossier
for fichier in fichiers:
    if fichier.endswith('.lab'):  # Assurez-vous que le fichier est au format LAB
        chemin_fichier = os.path.join(dossier, fichier)
        
        # Lisez le fichier LAB et créez un DataFrame
        df_temp = pd.read_csv(chemin_fichier, delimiter=' ', header=None, names=['start', 'end', 'chords'])
        
        # Ajoutez une colonne avec le nom du fichier
        df_temp['NomFichier'] = fichier[5:]
        
        # Concaténez le DataFrame temporaire au DataFrame final
        df_final = pd.concat([df_final, df_temp], ignore_index=True)

# Affichez le DataFrame final
df_final.shape

(1011, 4)

**Rangement des données propres dans le df**

In [46]:
new_df = df_final
# Affichez les premières lignes du nouveau DataFrame
new_df.head()

,start,end,chords,NomFichier
0,0.000000,2.612267,N,I_Saw_Her_Standing_There.lab
1,2.612267,11.459070,E,I_Saw_Her_Standing_There.lab
2,11.459070,12.921927,A,I_Saw_Her_Standing_There.lab
3,12.921927,17.443474,E,I_Saw_Her_Standing_There.lab
4,17.443474,20.410362,B,I_Saw_Her_Standing_There.lab


In [47]:
new_df['Decompo'] = concat(get_fund(new_df['chords']),get_type(new_df['chords']),get_basse_mod(new_df['chords']))
new_df.head()

,start,end,chords,NomFichier,Decompo
0,0.000000,2.612267,N,I_Saw_Her_Standing_There.lab,"[N, M, ]"
1,2.612267,11.459070,E,I_Saw_Her_Standing_There.lab,"[E, M, ]"
2,11.459070,12.921927,A,I_Saw_Her_Standing_There.lab,"[A, M, ]"
3,12.921927,17.443474,E,I_Saw_Her_Standing_There.lab,"[E, M, ]"
4,17.443474,20.410362,B,I_Saw_Her_Standing_There.lab,"[B, M, ]"


In [48]:
new_df['Good_chords'] = pretitify(new_df['Decompo'])
new_df.head()

,start,end,chords,NomFichier,Decompo,Good_chords
0,0.000000,2.612267,N,I_Saw_Her_Standing_There.lab,"[N, M, ]",NM
1,2.612267,11.459070,E,I_Saw_Her_Standing_There.lab,"[E, M, ]",EM
2,11.459070,12.921927,A,I_Saw_Her_Standing_There.lab,"[A, M, ]",AM
3,12.921927,17.443474,E,I_Saw_Her_Standing_There.lab,"[E, M, ]",EM
4,17.443474,20.410362,B,I_Saw_Her_Standing_There.lab,"[B, M, ]",BM


In [49]:
np.unique(np.array(new_df['Good_chords']))

array(['A7', 'AM', 'Amaj7', 'Amaj7/E', 'Amin', 'Amin/C', 'B7', 'BM',
       'Bb7', 'BbM', 'Bmaj6', 'Bmin', 'C#7', 'C#min', 'C#min/G#', 'CM',
       'Cmin6', 'D9', 'DM', 'DM/E', 'DM/F#', 'E7', 'E7/G#', 'E9', 'EM',
       'Eaug', 'Eb7', 'Eb9', 'EbM', 'Ebmin', 'Emaj7', 'Emin', 'F#9',
       'F#M', 'F#min', 'F#min7', 'F#minmaj7', 'F9', 'FM', 'G#M', 'G#min',
       'G#min7', 'GM', 'Gmin', 'NM'], dtype=object)

**Ajout des séquences de notes**

In [50]:
from mingus.core import chords
from mingus.containers import NoteContainer
from mingus.midi import midi_file_out as MidiFileOut

In [51]:
new_df['notes'] = new_df['Good_chords'].apply(lambda x: chords.from_shorthand(x) if 'N' not in x else np.nan)
new_df.head()

,start,end,chords,NomFichier,Decompo,Good_chords,notes
0,0.000000,2.612267,N,I_Saw_Her_Standing_There.lab,"[N, M, ]",NM,NaN
1,2.612267,11.459070,E,I_Saw_Her_Standing_There.lab,"[E, M, ]",EM,"[E, G#, B]"
2,11.459070,12.921927,A,I_Saw_Her_Standing_There.lab,"[A, M, ]",AM,"[A, C#, E]"
3,12.921927,17.443474,E,I_Saw_Her_Standing_There.lab,"[E, M, ]",EM,"[E, G#, B]"
4,17.443474,20.410362,B,I_Saw_Her_Standing_There.lab,"[B, M, ]",BM,"[B, D#, F#]"


In [52]:
# Utilisez la fonction apply de pandas avec len pour calculer la longueur de chaque séquence
new_df['taille_sequence'] = new_df['notes'].apply(lambda x: len(x) if isinstance(x, list) else np.nan)
new_df.head()

,start,end,chords,NomFichier,Decompo,Good_chords,notes,taille_sequence
0,0.000000,2.612267,N,I_Saw_Her_Standing_There.lab,"[N, M, ]",NM,NaN,NaN
1,2.612267,11.459070,E,I_Saw_Her_Standing_There.lab,"[E, M, ]",EM,"[E, G#, B]",3.0
2,11.459070,12.921927,A,I_Saw_Her_Standing_There.lab,"[A, M, ]",AM,"[A, C#, E]",3.0
3,12.921927,17.443474,E,I_Saw_Her_Standing_There.lab,"[E, M, ]",EM,"[E, G#, B]",3.0
4,17.443474,20.410362,B,I_Saw_Her_Standing_There.lab,"[B, M, ]",BM,"[B, D#, F#]",3.0


In [53]:
np.unique(np.array(new_df['taille_sequence']))

array([ 3.,  4.,  5., nan])

In [54]:
# Les listes ne sont pas prise sen charge dans sql

new_df['Decompo'] = new_df['Decompo'].astype(str)
new_df['notes'] = new_df['notes'].astype(str)
new_df.head()

,start,end,chords,NomFichier,Decompo,Good_chords,notes,taille_sequence
0,0.000000,2.612267,N,I_Saw_Her_Standing_There.lab,"['N', 'M', ' ']",NM,nan,NaN
1,2.612267,11.459070,E,I_Saw_Her_Standing_There.lab,"['E', 'M', ' ']",EM,"['E', 'G#', 'B']",3.0
2,11.459070,12.921927,A,I_Saw_Her_Standing_There.lab,"['A', 'M', ' ']",AM,"['A', 'C#', 'E']",3.0
3,12.921927,17.443474,E,I_Saw_Her_Standing_There.lab,"['E', 'M', ' ']",EM,"['E', 'G#', 'B']",3.0
4,17.443474,20.410362,B,I_Saw_Her_Standing_There.lab,"['B', 'M', ' ']",BM,"['B', 'D#', 'F#']",3.0


**Rangement des informations dans une base de donnée sql**

In [55]:
# Connexion à la base de données SQLite (ou création si elle n'existe pas)
conn = sqlite3.connect('ma_base_de_donnees.db')

# Itération sur chaque valeur unique dans la colonne 'NomFichier'
for nom_fichier in new_df['NomFichier'].unique():
    # Filtrer le DataFrame pour la valeur de 'NomFichier' actuelle
    subset_df = new_df[new_df['NomFichier'] == nom_fichier]

    # Création de la table correspondante dans la base de données
    subset_df.to_sql(nom_fichier, conn, index=False, if_exists='replace')  # Utilisez 'replace' ou 'append' selon votre besoin

# Fermeture de la connexion à la base de données
conn.close()

In [56]:
import sqlite3

# Connectez-vous à votre base de données SQLite
conn = sqlite3.connect('ma_base_de_donnees.db')

# Créez un objet curseur pour exécuter des commandes SQL
cursor = conn.cursor()

# Exécutez la requête SQL pour obtenir la liste des tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Récupérez les résultats de la requête
tables = cursor.fetchall()

# Affichez les noms des tables
for table in tables:
    print(table[0])

# Fermez le curseur et la connexion
cursor.close()
conn.close()


I_Saw_Her_Standing_There.lab
Misery.lab
Anna_(Go_To_Him).lab
Chains.lab
Boys.lab
Ask_Me_Why.lab
Please_Please_Me.lab
Love_Me_Do.lab
P._S._I_Love_You.lab
Baby_It's_You.lab
Do_You_Want_To_Know_A_Secret.lab
A_Taste_Of_Honey.lab
There's_A_Place.lab
Twist_And_Shout.lab


In [57]:
# import sqlite3

# # Connexion à la base de données SQLite
# conn = sqlite3.connect('ma_base_de_donnees.db')
# cursor = conn.cursor()

# # Obtenez la liste des tables dans la base de données
# cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = cursor.fetchall()

# # Parcourez la liste des tables et exécutez DROP TABLE pour chacune
# for table in tables:
#     table_name = table[0]
#     cursor.execute(f"DROP TABLE IF EXISTS [{table_name}];")

# # Validez la transaction
# conn.commit()

# # Fermeture de la connexion
# conn.close()